In [1]:
import os
from path import Path
import pickle
import numpy as np
import pandas as pd
import spacy
import seaborn as sns; sns.set()

In [2]:
!python -m spacy download en_core_web_md

/Users/josemonagas/anaconda3/envs/PythonData/bin/python: No module named spacy


In [3]:
data = 'Resources/mbti_clean.csv'
df = pd.read_csv(data)
df.head()

,type,posts
0,INFJ,The last thing INFJ friend posted his facebo...
1,INFJ,Hello ENFJ Sorry hear your distress Its only...
2,INFJ,Prozac wellbrutin least thirty minutes movin...
3,INFJ,Basically come with three items youve determi...
4,INFJ,All things moderation Sims indeed video ga...


In [4]:
embedding_dim = 300
X = np.empty((df.shape[0], embedding_dim))

In [5]:
def doc2vec(document: str):
    nlp = spacy.load("en_core_web_md")
    doc = nlp(document)
    return doc.vector

In [6]:
# for i, doc in enumerate(df["posts"]):
#     if i % 1000 == 0:
#         print(f"Completed {i} conversions")
#     X[i] = doc2vec(doc)

In [7]:
import pickle

with open("embeddings.pkl", "wb") as f:
    pickle.dump(X, f)

In [8]:
if os.path.exists("embeddings.pkl"):
    with open("embeddings.pkl", "rb") as f:
        X = pickle.load(f)
else:
    for i, doc in enumerate(df["posts"]):
        if i % 1000 == 0:
            print(f"Completed {i} conversions")
        X[i] = doc2vec(doc)

In [9]:
y = df["type"]

In [10]:
X = X[0 : 1000] 
y = y[0 : 1000]

In [11]:
# Split the dataset into training and testing sets

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,
   y,  random_state=1, stratify=y)
X_train.shape

(750, 300)

In [12]:
from sklearn.svm import SVC
model = SVC(kernel='linear')

In [13]:
# Train the model
model.fit(X_train, y_train)

SVC(kernel='linear')

In [14]:
# Create predictions with the model
y_pred = model.predict(X_test)
results = pd.DataFrame({
   "Prediction": y_pred,
   "Actual": y_test
}).reset_index(drop=True)
results.head()

,Prediction,Actual
0,INFJ,INTP
1,INFJ,ENTJ
2,INFJ,INFP
3,INFJ,ENTJ
4,INFJ,INFJ


In [15]:
# Assess the accuracy_score of the model
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.316

In [16]:
# Generate confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

        ENFJ       0.00      0.00      0.00         5
        ENFP       0.00      0.00      0.00        12
        ENTJ       0.00      0.00      0.00        21
        ENTP       0.00      0.00      0.00         7
        INFJ       0.32      1.00      0.48        79
        INFP       0.00      0.00      0.00        51
        INTJ       0.00      0.00      0.00        34
        INTP       0.00      0.00      0.00        36
        ISFP       0.00      0.00      0.00         5

    accuracy                           0.32       250
   macro avg       0.04      0.11      0.05       250
weighted avg       0.10      0.32      0.15       250



/Users/josemonagas/anaconda3/envs/mlenv/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
mat = confusion_matrix(y_test, y_pred)
sns.heatmap(mat.T, square=True, annot=True, fmt='d', cbar=False,
            xticklabels=df.type,
            yticklabels=df.type)
plt.xlabel('true label')
plt.ylabel('predicted label');